# Phân tích các yếu tố lối sống ảnh hưởng đến tình trạng Béo phì

## I. Khám phá dữ liệu (EDA)
1. Chuẩn bị dữ liệu và sắp xếp thứ tự phân loại.
2. Phân tích đơn biến (Univariate Analysis).
3. Phân tích yếu tố xã hội và di truyền.
4. Phân tích thói quen và thể chất.
5. Phân tích tương quan.
6. Feature Engineering & Data Cleaning (BMI, Rounding).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import utils

# Cấu hình hiển thị
pd.set_option('display.max_columns', None)
%matplotlib inline

# Thiết lập style cho biểu đồ
sns.set(style="whitegrid")

In [ ]:
# Load dữ liệu
df = pd.read_csv('ObesityDataSet_raw_and_data_sinthetic.csv')
print(f'Kích thước tập dữ liệu: {df.shape}')
df.head()

### 1. Chuẩn bị: Sắp xếp thứ tự tự nhiên của dữ liệu
Dữ liệu béo phì có tính thứ tự (Ordinal). Ta sẽ sử dụng danh sách thứ tự đã định nghĩa trong `utils.py` để các biểu đồ hiển thị logic hơn.

In [ ]:
print("Thứ tự phân loại béo phì:")
print(utils.OBESITY_ORDER)

### 2. Phân tích Đơn biến (Univariate Analysis)
Kiểm tra sự cân bằng của biến mục tiêu `NObeyesdad`.

In [ ]:
plt.figure(figsize=(12, 6))
sns.countplot(y='NObeyesdad', data=df, order=utils.OBESITY_ORDER, palette='viridis', hue='NObeyesdad', legend=False)
plt.title('Phân phối các mức độ béo phì')
plt.xlabel('Số lượng')
plt.ylabel('Mức độ béo phì')
plt.show()

### 3. Phân tích yếu tố Xã hội & Di truyền
Xem xét mối quan hệ giữa `family_history_with_overweight`, `MTRANS` với `NObeyesdad`.

In [ ]:
plt.figure(figsize=(12, 6))
sns.countplot(y='NObeyesdad', hue='family_history_with_overweight', data=df, order=utils.OBESITY_ORDER, palette='Set2')
plt.title('Tiền sử gia đình thừa cân vs Mức độ béo phì')
plt.legend(title='Tiền sử gia đình', loc='lower right')
plt.show()

In [ ]:
plt.figure(figsize=(14, 8))
sns.countplot(x='NObeyesdad', hue='MTRANS', data=df, order=utils.OBESITY_ORDER, palette='Paired')
plt.title('Phương tiện di chuyển vs Mức độ béo phì')
plt.xticks(rotation=45)
plt.legend(title='Phương tiện', loc='upper right')
plt.show()

### 4. Phân tích thói quen và Thể chất
Sử dụng Boxplot để xem xét phân phối của `Age` và `FAF` (Physical Activity Frequency) theo từng nhóm béo phì.

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(x='NObeyesdad', y='Age', data=df, order=utils.OBESITY_ORDER, palette='coolwarm', hue='NObeyesdad', legend=False)
plt.title('Phân phối Tuổi tác theo Mức độ béo phì')
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(x='NObeyesdad', y='FAF', data=df, order=utils.OBESITY_ORDER, palette='coolwarm', hue='NObeyesdad', legend=False)
plt.title('Tần suất hoạt động thể chất (FAF) theo Mức độ béo phì')
plt.xticks(rotation=45)
plt.show()

### 5. Phân tích Tương quan (Correlation Heatmap)
Chuyển đổi dữ liệu categorical sang dạng số để vẽ Heatmap.

In [ ]:
# Tạo bản sao và encode dữ liệu
df_encoded = df.copy()
le = LabelEncoder()
for col in df_encoded.select_dtypes(include=['object']).columns:
    df_encoded[col] = le.fit_transform(df_encoded[col])

# Tính ma trận tương quan
plt.figure(figsize=(12, 10))
sns.heatmap(df_encoded.corr(), annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Biểu đồ nhiệt tương quan giữa các biến')
plt.show()

### 6. Feature Engineering & Data Cleaning

1. Tính chỉ số BMI.
2. Kiểm tra tính nhất quán của dữ liệu (BMI vs NObeyesdad).
3. Làm mịn dữ liệu (Rounding) cho các biến số thực.

#### 6.1. Feature Engineering: Tính chỉ số BMI
Công thức: $BMI = Weight / (Height^2)$

In [ ]:
df['BMI'] = df['Weight'] / (df['Height'] ** 2)
print('5 dòng đầu tiên của cột BMI:')
print(df[['Height', 'Weight', 'BMI']].head())

#### 6.2. Kiểm tra tính nhất quán (BMI vs NObeyesdad)
Vẽ biểu đồ Boxplot của BMI theo từng nhóm béo phì để xem sự phân bố có tuân theo quy luật tăng dần hay không.

In [ ]:
plt.figure(figsize=(14, 8))
sns.boxplot(x='NObeyesdad', y='BMI', data=df, order=utils.OBESITY_ORDER, palette='Spectral', hue='NObeyesdad', legend=False)
plt.title('Phân phối BMI theo Mức độ béo phì')
plt.xticks(rotation=45)

# Vẽ các đường giới hạn chuẩn BMI (tham khảo WHO)
plt.axhline(y=18.5, color='r', linestyle='--', label='Underweight (<18.5)')
plt.axhline(y=25, color='orange', linestyle='--', label='Overweight (>=25)')
plt.axhline(y=30, color='yellow', linestyle='--', label='Obesity (>=30)')
plt.legend()
plt.show()

#### 6.3. Làm tròn dữ liệu (Rounding)
Các biến như `Age`, `FCVC`, `NCP`, `CH2O`, `FAF`, `TUE` trong dữ liệu tổng hợp (synthetic) có dạng số thực. Ta sẽ làm tròn chúng về số nguyên để dễ phân tích.

In [ ]:
columns_to_round = ['Age', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE']

print("Trước khi làm tròn:")
print(df[columns_to_round].head())

for col in columns_to_round:
    df[col] = df[col].round().astype(int)

print("\nSau khi làm tròn:")
print(df[columns_to_round].head())

## II. Tiền xử lý dữ liệu (Advanced Preprocessing)
1. Xử lý các biến có thứ tự (Ordinal Encoding) cho `CAEC` và `CALC`.
2. Xử lý biến danh mục bằng One-Hot Encoding cho `MTRANS` và `Gender`.
3. Mã hóa biến mục tiêu (Label Encoding) cho `NObeyesdad`.
4. Chuẩn hóa dữ liệu số (Feature Scaling) sử dụng StandardScaler.

### 1. Xử lý các biến có thứ tự (Ordinal Encoding)
Các cột `CAEC` (Ăn vặt) và `CALC` (Uống rượu) chứa các giá trị có thứ tự tự nhiên: no < Sometimes < Frequently < Always.

In [ ]:
# Định nghĩa mapping
ordinal_mapping = {
    'no': 0,
    'Sometimes': 1,
    'Frequently': 2,
    'Always': 3
}

# Áp dụng mapping
df['CAEC'] = df['CAEC'].map(ordinal_mapping)
df['CALC'] = df['CALC'].map(ordinal_mapping)

print("Kiểm tra sau khi Ordinal Encoding:")
print(df[['CAEC', 'CALC']].head())

### 2. Xử lý biến danh mục bằng One-Hot Encoding
Dành cho các biến không có thứ tự hơn kém như `MTRANS` (Phương tiện), `Gender`, `family_history_with_overweight`, `FAVC`, `SMOKE`, `SCC`.

In [ ]:
# Các cột cần One-Hot Encoding
# Lưu ý: family_history_with_overweight, FAVC, SMOKE, SCC là binary (yes/no), 
# nhưng ta cũng có thể dùng One-Hot hoặc Label Encoding (vì chỉ có 2 giá trị 0/1).
# Ở đây ta dùng pd.get_dummies cho tất cả các biến danh mục còn lại (trừ NObeyesdad)

categorical_cols = ['Gender', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC', 'MTRANS']

df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)
# drop_first=True để tránh bẫy đa cộng tuyến (Dummy Variable Trap)

print("Kích thước dữ liệu sau khi One-Hot Encoding:", df.shape)
df.head()

### 3. Mã hóa biến mục tiêu (Label Encoding)
Chuyển đổi cột `NObeyesdad` từ chuỗi sang số nguyên.

In [ ]:
from sklearn.preprocessing import LabelEncoder

target_le = LabelEncoder()
df['NObeyesdad'] = target_le.fit_transform(df['NObeyesdad'])

print("Mapping của biến mục tiêu:")
for i, label in enumerate(target_le.classes_):
    print(f"{label} -> {i}")

### 4. Chuẩn hóa dữ liệu số (Feature Scaling)
Sử dụng `StandardScaler` để đưa các biến số về cùng một tỷ lệ (mean=0, std=1).
Các biến số cần chuẩn hóa: `Age`, `Height`, `Weight`, `BMI`, `FCVC`, `NCP`, `CH2O`, `FAF`, `TUE`.

In [ ]:
from sklearn.preprocessing import StandardScaler

numerical_cols = ['Age', 'Height', 'Weight', 'BMI', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE']

scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

print("Dữ liệu sau khi chuẩn hóa (5 dòng đầu):")
print(df[numerical_cols].head())

## III. Xây dựng và Đánh giá Mô hình
1. Chia dữ liệu thành tập Train và Test.
2. Huấn luyện 3 mô hình khác nhau: Random Forest, SVM, và KNN.
3. Đánh giá mô hình bằng Accuracy, Classification Report và Confusion Matrix.
4. Tối ưu hóa tham số (Hyperparameter Tuning) cho Random Forest.
5. Phân tích các yếu tố quan trọng (Feature Importance).

### 1. Chia tập Train - Test Split
Tỷ lệ chia: 80% Train - 20% Test.
Random State: 42 (để đảm bảo tính tái lập).

In [ ]:
from sklearn.model_selection import train_test_split

# Tách features và target
X = df.drop('NObeyesdad', axis=1)
y = df['NObeyesdad']

# Chia dữ liệu
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Kích thước tập Train: {X_train.shape}")
print(f"Kích thước tập Test: {X_test.shape}")

### 2 & 3. Huấn luyện và Đánh giá 3 Mô hình cơ sở
Các mô hình được thử nghiệm:
- **Random Forest**: Mô hình mạnh mẽ cho dữ liệu bảng.
- **SVM**: Hiệu quả với không gian nhiều chiều.
- **KNN**: Thuật toán đơn giản dựa trên khoảng cách.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import importlib
import utils
importlib.reload(utils)

# Khởi tạo các mô hình
models = {
    "Random Forest": RandomForestClassifier(random_state=42),
    "SVM": SVC(random_state=42),
    "KNN": KNeighborsClassifier()
}

# Lưu kết quả
results = {}
target_names = target_le.classes_

for name, model in models.items():
    print(f"--- Đang huấn luyện {name} ---")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    results[name] = acc
    print(f"Accuracy: {acc:.4f}")
    print(f"Classification Report:\n{classification_report(y_test, y_pred, target_names=target_names)}")
    
    # Vẽ Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    utils.plot_confusion_matrix(cm, classes=target_names, title=f'Confusion Matrix - {name}')

### 4. Tối ưu hóa (Hyperparameter Tuning) cho Random Forest
Sử dụng GridSearchCV để tìm bộ tham số tốt nhất.

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}

rf_grid = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=3, n_jobs=-1, verbose=1)
rf_grid.fit(X_train, y_train)

print(f"Best Params: {rf_grid.best_params_}")
print(f"Best Score (CV): {rf_grid.best_score_:.4f}")

best_rf = rf_grid.best_estimator_
y_pred_best = best_rf.predict(X_test)
print(f"Optimized Accuracy (Test): {accuracy_score(y_test, y_pred_best):.4f}")

### 5. Phân tích Nhân tố quan trọng (Feature Importance)
Xem xét mô hình Random Forest dựa vào đặc trưng nào để phân loại.

In [ ]:
importances = best_rf.feature_importances_
feature_names = X.columns

utils.plot_feature_importance(importances, feature_names, top_n=10)

### 6. Dự đoán chỉ dựa trên Lối sống (Bỏ BMI, Height, Weight)
Vì BMI được tính trực tiếp từ Height và Weight, nên việc mô hình dự đoán chính xác khi có các biến này là điều dễ hiểu.
Việc này sẽ loại bỏ các chỉ số cơ thể để xem liệu mô hình có thể dự đoán tình trạng béo phì chỉ dựa trên thói quen ăn uống, vận động và yếu tố xã hội hay không.

In [ ]:
# Loại bỏ các cột chỉ số cơ thể
X_lifestyle = df.drop(['NObeyesdad', 'BMI', 'Height', 'Weight'], axis=1)
y = df['NObeyesdad']

# Scaling lại (vì X_lifestyle chỉ còn một số biến số thực)
scaler_life = StandardScaler()
num_cols_life = ['Age', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE']
X_lifestyle[num_cols_life] = scaler_life.fit_transform(X_lifestyle[num_cols_life])

# Chia tập dữ liệu
X_train_life, X_test_life, y_train_life, y_test_life = train_test_split(X_lifestyle, y, test_size=0.2, random_state=42)

print(f"Kích thước tập Train (Lifestyle): {X_train_life.shape}")
print(f"Các features sử dụng: {list(X_train_life.columns)}")

# Huấn luyện Random Forest (sử dụng best params từ bước trước)
rf_lifestyle = RandomForestClassifier(random_state=42, n_estimators=200)
rf_lifestyle.fit(X_train_life, y_train_life)

# Đánh giá
y_pred_life = rf_lifestyle.predict(X_test_life)
acc_life = accuracy_score(y_test_life, y_pred_life)

print(f"\nAccuracy (Lifestyle Only): {acc_life:.4f}")
print(classification_report(y_test_life, y_pred_life, target_names=target_names))

# Vẽ Confusion Matrix
cm_life = confusion_matrix(y_test_life, y_pred_life)
utils.plot_confusion_matrix(cm_life, classes=target_names, title='Confusion Matrix - Lifestyle Only')

# Feature Importance
imp_life = rf_lifestyle.feature_importances_
utils.plot_feature_importance(imp_life, X_lifestyle.columns, top_n=10)